In [2]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense,Conv2D,MaxPooling2D,Add
from keras import layers
import tensorflow as tf

In [4]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("shiekhburhan/face-mask-dataset")

print("Path to dataset files:", path)

100%|██████████| 2.33G/2.33G [03:55<00:00, 10.6MB/s]

Extracting files...


Path to dataset files: C:\Users\Asus\.cache\kagglehub\datasets\shiekhburhan\face-mask-dataset\versions\1


In [5]:
datagen = ImageDataGenerator(rescale=1/255, validation_split=0.8)
data    = datagen.flow_from_directory("./FMD_DATASET")
data.class_indices

Found 14536 images belonging to 3 classes.


{'incorrect_mask': 0, 'with_mask': 1, 'without_mask': 2}

In [8]:
train_dataset = datagen.flow_from_directory(batch_size=32,
                                                 directory='./FMD_DATASET',
                                                 shuffle=True,
                                                 target_size=(280, 280), 
                                                 subset="training",
                                                 class_mode='categorical')

Found 2908 images belonging to 3 classes.


In [11]:
#set device
import tensorflow as tf
print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        # เลือกใช้ GPU ตัวแรก
        tf.config.set_visible_devices(gpus[0], 'GPU')
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        print(e)



Num GPUs Available: 0


In [12]:
model = Sequential()
model.add(layers.Input(shape = data.image_shape))
model.add(layers.Conv2D(32, 3, padding ='same', activation = 'relu'))
#model.add(layers.Dropout(0.1))
model.add(layers.MaxPooling2D())
#model.add(layers.Dropout(0.2))
model.add(layers.Conv2D(64, 3, padding = 'same', activation = 'relu'))
model.add(layers.MaxPooling2D())
model.add(layers.Flatten())
#model.add(layers.Dense(128,activation = 'relu'))
model.add(layers.Dense(3, activation = 'softmax'))

model.compile(
optimizer = tf.keras.optimizers.Adam(learning_rate = 0.005),
loss = tf.keras.losses.CategoricalCrossentropy(),
metrics = ['accuracy'])

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 256, 256, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 128, 128, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 128, 128, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 64, 64, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 262144)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 3)              │       786,435 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 805,827 (3.07 MB)

 Trainable params: 805,827 (3.07 MB)

 Non-trainable params: 0 (0.00 B)

In [13]:
history = model.fit(data, epochs = 20,validation_batch_size=0.15)

c:\Users\Asus\facemask_detection\mask_detect\.venv\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
357/455 ━━━━━━━━━━━━━━━━━━━━ 1:36 987ms/step - accuracy: 0.7591 - loss: 2.5042

c:\Users\Asus\facemask_detection\mask_detect\.venv\lib\site-packages\PIL\Image.py:970: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


455/455 ━━━━━━━━━━━━━━━━━━━━ 445s 975ms/step - accuracy: 0.8733 - loss: 0.6600
Epoch 2/20
455/455 ━━━━━━━━━━━━━━━━━━━━ 286s 629ms/step - accuracy: 0.9210 - loss: 0.2359
Epoch 3/20
455/455 ━━━━━━━━━━━━━━━━━━━━ 286s 628ms/step - accuracy: 0.9088 - loss: 0.2913
Epoch 4/20
455/455 ━━━━━━━━━━━━━━━━━━━━ 282s 619ms/step - accuracy: 0.9119 - loss: 0.2606
Epoch 5/20
455/455 ━━━━━━━━━━━━━━━━━━━━ 281s 618ms/step - accuracy: 0.9362 - loss: 0.1908
Epoch 6/20
455/455 ━━━━━━━━━━━━━━━━━━━━ 281s 618ms/step - accuracy: 0.9458 - loss: 0.1575
Epoch 7/20
455/455 ━━━━━━━━━━━━━━━━━━━━ 280s 614ms/step - accuracy: 0.9550 - loss: 0.1344
Epoch 8/20
455/455 ━━━━━━━━━━━━━━━━━━━━ 280s 615ms/step - accuracy: 0.9582 - loss: 0.1217
Epoch 9/20
455/455 ━━━━━━━━━━━━━━━━━━━━ 280s 615ms/step - accuracy: 0.9642 - loss: 0.1060
Epoch 10/20
455/455 ━━━━━━━━━━━━━━━━━━━━ 280s 615ms/step - accuracy: 0.9686 - loss: 0.0863
Epoch 11/20
455/455 ━━━━━━━━━━━━━━━━━━━━ 279s 613ms/step - accuracy: 0.9754 - loss: 0.0761
Epoch 12/20
455/455

In [15]:
#save model
model.save('mask_detector_model2.h5')